# Statistical Analysis for Dark Higgs

- The idea is to convert the below bar chart to an p-value and then make an exclusion contour

![DarkHiggsYeilds](assets/DarkHiggsYields.png)

In [1]:
import ROOT
import json
print(ROOT.gROOT.GetVersion(),ROOT.xRooFit.GetVersion())

Warning in cling::IncrementalParser::CheckABICompatibility():
  Possible C++ standard library mismatch, compiled with __GLIBCXX__ '20250425'
  Extraction of runtime standard library version was: '20250521'


xRooFit -- Create/Explore/Modify Workspaces -- Development ongoing 
                xRooFit : http://gitlab.cern.ch/will/xroofit
                Version: v0.0.1-14-g1b8b81e [2025-06-09 15:42:36 +0100]
6.37.01 v0.0.1-14-g1b8b81e


In [2]:
signal_yields = json.load(open("dark_higgs_yield.json"))

bkg_yield = {
    "nevents": 0.0304,
    "nevents_err": 0.004387862045841156
}

data_obs = {
    "nevents": 0.0,
    "nevents_err": 0.0
}

print(signal_yields["dh_113900"])

{'mass': 0.23, 'coupling': 0.0007, 'crosssection': 2.246042502785529e-05, 'sumw': 20000, 'nevents': 1.4212844655501689, 'nevents_err': 0.01741451134044757}


In [3]:
def make_histogram(name, yields):
    nevents = yields["nevents"]
    nevents_err = yields["nevents_err"]
    hist = ROOT.TH1D(name, name, 1, 0, 1)
    hist.SetBinContent(1, nevents)
    hist.SetBinError(1, nevents_err)
    hist.SetXTitle("SR")
    hist.SetYTitle("Events")
    return hist

In [4]:
# Make a workspace
w = ROOT.xRooNode("RooWorkspace", "combined", "my workspace")

h_bkg = make_histogram("bkg", bkg_yield)
h_data = make_histogram("obsData", data_obs)
h_sigs = {}

w["pdfs/simPdf/SR"].Add(h_bkg)
w["pdfs/simPdf/SR"].datasets().Add(h_data)
for sig_name, sig_yield in signal_yields.items():
    h_sig = make_histogram(sig_name, sig_yield)
    h_sigs[sig_name] = h_sig
    w["pdfs/simPdf/SR"].Add(h_sigs[sig_name])

[#1] INFO:ObjectHandling -- RooWorkspace::import(combined) importing dataset obsData


Info in <xRooNode::Add>: Created pdf RooSimultaneous::simPdf in workspace combined
Info in <xRooNode::Vary>: Created channel RooProdPdf::simPdf_SR in model simPdf
Info in <xRooNode::Multiply>: Created RooRealSumPdf::simPdf_SR_samples in channel simPdf_SR
Info in <xRooNode::Multiply>: Created Shape factor simPdf_SR_samples_bkg_statFactor in prod_simPdf_SR_samples_bkg
Info in <xRooNode::Constrain>: Added poisson constraint pdf RooPoisson::pois_stat_simPdf_SR_samples_bin1 (tau=48.64) for stat_simPdf_SR_samples_bin1
Info in <xRooNode::Add>: Created SimpleDensity factor RooHistFunc::simPdf_SR_samples_bkg for simPdf_SR_samples
Info in <xRooNode::Multiply>: Created Shape factor simPdf_SR_samples_dh_113900_statFactor in prod_simPdf_SR_samples_dh_113900
Info in <xRooNode::Add>: Created SimpleDensity factor RooHistFunc::simPdf_SR_samples_dh_113900 for simPdf_SR_samples
Info in <xRooNode::Multiply>: Created Shape factor simPdf_SR_samples_dh_113901_statFactor in prod_simPdf_SR_samples_dh_113901
In

In [5]:
w["pdfs/simPdf/SR"].Print("depth=3")

combined/simPdf/channelCat=SR: RooProdPdf::simPdf_SR
0) samples : RooRealSumPdf::simPdf_SR_samples
 0) bkg : RooProduct::prod_simPdf_SR_samples_bkg
  0) bkg : RooHistFunc::simPdf_SR_samples_bkg [SimpleDensity]
   0) xaxis : RooRealVar::xaxis = 0.5  L(0 - 1) B(1) 
  1) statFactor : ParamHistFunc::simPdf_SR_samples_bkg_statFactor [Shape]
   0) stat_simPdf_SR_samples_bin1 : RooRealVar::stat_simPdf_SR_samples_bin1 = 1 +/- 0.0034946  L(0.982527 - 1.01747) 
 1) dh_113900 : RooProduct::prod_simPdf_SR_samples_dh_113900
  0) dh_113900 : RooHistFunc::simPdf_SR_samples_dh_113900 [SimpleDensity]
   0) xaxis : RooRealVar::xaxis = 0.5  L(0 - 1) B(1) 
  1) statFactor : ParamHistFunc::simPdf_SR_samples_dh_113900_statFactor [Shape]
   0) stat_simPdf_SR_samples_bin1 : RooRealVar::stat_simPdf_SR_samples_bin1 = 1 +/- 0.0034946  L(0.982527 - 1.01747) 
 2) dh_113901 : RooProduct::prod_simPdf_SR_samples_dh_113901
  0) dh_113901 : RooHistFunc::simPdf_SR_samples_dh_113901 [SimpleDensity]
   0) xaxis : RooRealV

In [6]:
w["pdfs/simPdf/SR"].nll("obsData").hypoSpace().scan("cls")

runtime_error: int xRooNLLVar::xRooHypoSpace::scan(const char* type = "cls", const vector<double>& nSigmas = {0, 1, 2, -1, -2, std::numeric_limits<double>::quiet_NaN()}, double relUncert = 0.10000000000000001) =>
    runtime_error: No POI to scan